In [15]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np

# Step 2: Function to calculate land use proportions
def calculate_lulc_proportions(lulc_raster, point, buffer_radius):
    """
    Calculate proportions of LULC categories within a circular buffer around a point.
    """

    # point_projected = point.to_crs(epsg=32633)  # UTM zone 33N
    # buffer = point_projected.buffer(buffer_radius)
    # buffer_original = buffer.to_crs(epsg=4326)

    buffer = point.buffer(buffer_radius, resolution=50)  # Buffer with high resolution

    # Create a circular buffer around the point (convert to GeoJSON format)
    geojson_buffer = [buffer.__geo_interface__]  # Convert to GeoJSON format for rasterio.mask

    # Mask the LULC raster using the buffer
    try:
        out_image, out_transform = mask(lulc_raster, geojson_buffer, crop=True)
        data = out_image[0]  # Extract the first band
        data = data[data > 0]  # Remove invalid or no-data values (assumed to be <= 0)

        # Calculate proportions of each LULC category
        unique, counts = np.unique(data, return_counts=True)
        proportions = {f"lulc_{int(cat)}": round(count/data.size, 3) for cat, count in zip(unique, counts)}
        return proportions
    except Exception as e:
        print(f"Error processing buffer at point {point}: {e}")
        return {}


In [19]:
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# Function to extract lat, lon for each pixel
def extract_lat_lon_to_geopandas(tif_file):
    # Open the GeoTIFF file
    with rasterio.open(tif_file) as src:
        # Read the dataset metadata
        transform = src.transform
        crs = src.crs
        width = src.width
        height = src.height

        # Get pixel coordinates
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))

        # Convert pixel coordinates to geographic coordinates
        xs, ys = rasterio.transform.xy(transform, rows, cols)
        lons = np.array(xs).flatten()
        lats = np.array(ys).flatten()

        # Read the LST values
        # lst_data = src.read(1)  # Assuming LST is in the first band
        # lst_values = lst_data.flatten()

        # Create GeoPandas DataFrame
        data = {
            "latitude": lats,
            "longitude": lons,
            # "temperature": lst_values,
            "geometry": [Point(lon, lat) for lon, lat in zip(lons, lats)],
        }
        gdf = gpd.GeoDataFrame(data, crs=crs)

    return gdf


In [ ]:
# Path to the ECOSTRESS LST GeoTIFF file
tif_file = '../ecostress_LST/ECO2LSTE.001_SDS_LST_doy2024181064258_aid0001.tif'

# Create the GeoDataFrame
gdf = extract_lat_lon_to_geopandas(tif_file)

# Display the GeoDataFrame
print(gdf.head())

# Save to a GeoJSON file (optional)
# gdf.to_file("/path/to/save/ecostress_lst_points.geojson", driver="GeoJSON")
# print("GeoDataFrame saved to GeoJSON.")


In [ ]:


lst_data_file = '../downloads/LST_2022-8-10_2022-8-10.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(lst_data_file)

# Convert the 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Filter rows where 'hour' equals 0
df = df[df['hour'] == 0]

# Step 5: Convert to GeoPandas GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]  # Create Point geometries
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # Set CRS to WGS 84 (EPSG:4326)


In [ ]:

# Step 1: Load the LULC raster and the temperature GeoDataFrame
lulc_file = "ESRI_LULC_2023_Rome.tif"  # Path to the ESRI LULC TIFF file

# Buffer radius: approximately 3000 meters
# buffer_radius_3k = 0.03

# Buffer radius: approximately 40 meters
buffer_radius_35m = 0.00040

# Step 3: Process each point in the GeoDataFrame
with rasterio.open(lulc_file) as lulc_raster:
    all_proportions = []  # List to store proportions for each point
    for idx, row in gdf.iterrows():
        point = row.geometry  # Get the point geometry
        print(f'Point: {idx}: {point}')
        proportions = calculate_lulc_proportions(lulc_raster, point, buffer_radius_35m)
        all_proportions.append(proportions)

# Step 4: Merge LULC proportions with the GeoDataFrame
proportions_df = pd.DataFrame(all_proportions).fillna(0)  # Replace NaNs with 0 for missing LULC categories
gdf = pd.concat([gdf, proportions_df], axis=1)

In [ ]:
gdf.head()

In [ ]:
gdf.tail()

In [ ]:
# Assuming 'gdf' is your GeoDataFrame
if "lulc_2" in gdf.columns:
    print("Summary statistics for column 'lulc_2':")
    print(gdf["lulc_2"].describe())
else:
    print("'lulc_2' column not found in the GeoDataFrame.")

In [ ]:
gdf.shape

In [ ]:
# Count the number of rows where 'lulc_2' > 0
if "lulc_2" in gdf.columns:
    count = (gdf["lulc_2"] > 0).sum()
    print(f"Number of rows with 'lulc_2' > = {count}")
else:
    print("'lulc_2' column not found in the GeoDataFrame.")


In [ ]:
import matplotlib.pyplot as plt

# Check if 'lulc_2' column exists in the GeoDataFrame
if "lulc_2" in gdf.columns:
    # Plot histogram
    plt.figure(figsize=(10, 6))
    gdf["lulc_2"].hist(bins=30, edgecolor='black', color='skyblue')
    plt.title("Histogram of 'lulc_2'", fontsize=16)
    plt.xlabel("lulc_2", fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
else:
    print("'lulc_2' column not found in the GeoDataFrame.")


In [3]:
import rasterio
from rasterio.windows import from_bounds
import numpy as np
from scipy.ndimage import generic_filter
from rasterio.features import geometry_mask

# Define LULC remap dictionary
lulc_remap = {
    'lulc_0': 'no_data',
    'lulc_1': 'water',
    'lulc_2': 'trees',
    'lulc_4': 'flooded_veg',
    'lulc_5': 'crop',
    'lulc_7': 'built_area',
    'lulc_8': 'bare_ground',
    'lulc_9': 'snow_or_ice',
    'lulc_10': 'clouds',
    'lulc_11': 'range_land'
}

# Input file paths
lulc_path = "ESRI_LULC_2023_Rome.tif"
lst_path = "../ecostress_LST/ECO2LSTE.001_SDS_LST_doy2024181064258_aid0001.tif"
output_path = "ESRI_LULC_Proportions_Rome_Circle_40m.tif"


In [2]:
# Define circle radius in meters
circle_radius = 0.00040

# Read LULC raster
with rasterio.open(lulc_path) as lulc_raster:
    lulc_data = lulc_raster.read(1)
    lulc_transform = lulc_raster.transform
    lulc_crs = lulc_raster.crs
    lulc_resolution = lulc_transform[0]

# Read LST raster
with rasterio.open(lst_path) as lst_raster:
    lst_data = lst_raster.read(1)
    lst_transform = lst_raster.transform
    lst_crs = lst_raster.crs
    lst_resolution = lst_transform[0]
    lst_profile = lst_raster.profile

# Ensure both rasters share the same CRS
if lulc_crs != lst_crs:
    raise ValueError("LULC and LST rasters must have the same CRS.")

# Calculate kernel size for 40-meter radius in LULC resolution
kernel_size = int(2 * circle_radius / lulc_resolution) + 1
half_kernel = kernel_size // 2

# Create a circular kernel
y, x = np.ogrid[-half_kernel: half_kernel + 1, -half_kernel: half_kernel + 1]
circle_kernel = (x**2 + y**2) <= (circle_radius / lulc_resolution)**2

# Define a function to calculate proportions within the kernel
def calculate_proportions(lulc_window, unique_classes, circle_mask):
    proportions = np.zeros(len(unique_classes), dtype=np.float32)
    if lulc_window.shape == circle_mask.shape:
        for idx, lulc_class in enumerate(unique_classes):
            proportions[idx] = np.sum((lulc_window == lulc_class) & circle_mask) / np.sum(circle_mask)
    return proportions

# Initialize output array
unique_classes = np.unique(lulc_data)
proportions_stack = np.zeros((len(unique_classes), *lst_data.shape), dtype=np.float32)

# Loop through each LST pixel and calculate proportions
for row in range(lst_data.shape[0]):
    for col in range(lst_data.shape[1]):
        # Convert LST pixel center to geographic coordinates
        x, y = rasterio.transform.xy(lst_transform, row, col)
        # Get bounds for the 40-meter radius in LULC raster
        bounds = [
            x - circle_radius,
            y - circle_radius,
            x + circle_radius,
            y + circle_radius
        ]
        window = from_bounds(*bounds, transform=lulc_transform, height=kernel_size, width=kernel_size)
        lulc_window = lulc_data[
            int(window.row_off): int(window.row_off + window.height),
            int(window.col_off): int(window.col_off + window.width)
        ]
        if lulc_window.size > 0:
            # Calculate proportions for the pixel
            proportions_stack[:, row, col] = calculate_proportions(lulc_window, unique_classes, circle_kernel)

# Rename bands based on the LULC remap dictionary
band_names = [lulc_remap.get(f'lulc_{int(class_value)}', f'class_{int(class_value)}') for class_value in unique_classes]

# Update the profile for the output raster
lst_profile.update(count=proportions_stack.shape[0], dtype=rasterio.float32)


/home/emaximus/workspace/lsasaf_data_access/.venv/lib/python3.10/site-packages/rasterio/windows.py:314: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(


Proportional LULC raster saved to: LULC_Proportions_Rome_Circle_40m.tif


In [4]:

# Write output raster
with rasterio.open(output_path, 'w', **lst_profile) as dst:
    for idx, band_name in enumerate(band_names):
        dst.write(proportions_stack[idx, :, :], idx + 1)
        dst.set_band_description(idx + 1, band_name)

print("Proportional LULC raster saved to:", output_path)


Proportional LULC raster saved to: ESRI_LULC_Proportions_Rome_Circle_40m.tif
